<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [ ]:
# TODO - Words, words, mere words, no matter from the heart.
# create sonnet files using from a file shakes.txt (above Project Gutenberg text):
#     head -n 2907 shakes.txt |tail -n 2770 |sed "s/\’/_/"|split -d -a3 -l18 - sonnets.

In [39]:
# Read in Data From articles folder

import os, re

data_files = os.listdir('./shakes')

In [40]:
data_files[0]

'sonnets.109'

In [41]:
len(data_files)

697

In [42]:
sonnets = []
for filename in data_files:
    if filename.startswith('sonnets'): # make sure only loading the text files
        path = os.path.join('./shakes', filename)
        with open(path, 'r') as f:
            content = f.read()
            content = re.sub('[^A-Za-z0-9]+', ' ', content) #elminate non-alphanumerics
            sonnets.append(content)

In [43]:
sonnets[105]

' 63 Against my love shall be as I am now With Time s injurious hand crushed and o erworn When hours have drained his blood and filled his brow With lines and wrinkles when his youthful morn Hath travelled on to age s steepy night And all those beauties whereof now he s king Are vanishing or vanished out of sight Stealing away the treasure of his spring For such a time do I now fortify Against confounding age s cruel knife That he shall never cut from memory My sweet love s beauty though my lover s life His beauty shall in these black lines be seen And they shall live and he in them still green '

In [44]:
len(sonnets)

154

In [45]:
sonnet_text = []
sonnet_text = [a[:50] for a in sonnets]

In [46]:
# Based on "The Unreasonable Effectiveness of RNN" implementation
import numpy as np

chars = []
sonnet_text = " ".join(sonnet_text)
chars = list(set(sonnet_text)) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(sonnet_text)

print("unique characters : ", num_chars)
print("txt_data_size : ", txt_data_size)

unique characters :  54
txt_data_size :  7853


In [47]:
# one hot encode
char_to_int = dict((c, i) for i, c in enumerate(chars)) # "enumerate" returns index and value. Convert it to dictionary
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
integer_encoded = [char_to_int[i] for i in sonnet_text] # "integer_encoded" is a list which has a sequence converted from an original data to integers.
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

{'g': 0, '0': 1, 'B': 2, '3': 3, 'v': 4, 'H': 5, 'F': 6, '8': 7, 'P': 8, 'Y': 9, 'a': 10, 'n': 11, 'j': 12, '2': 13, 'C': 14, 'd': 15, 'L': 16, 'U': 17, '9': 18, 'W': 19, '1': 20, '6': 21, 'y': 22, 'T': 23, 'p': 24, 'A': 25, 's': 26, 'I': 27, 'S': 28, 'O': 29, 'u': 30, 'k': 31, 'c': 32, ' ': 33, 'f': 34, 'm': 35, 'x': 36, 'b': 37, 'K': 38, 't': 39, 'h': 40, 'M': 41, 'e': 42, 'o': 43, 'w': 44, 'l': 45, 'i': 46, '4': 47, '5': 48, 'N': 49, 'r': 50, '7': 51, 'D': 52, 'E': 53}
----------------------------------------------------
{0: 'g', 1: '0', 2: 'B', 3: '3', 4: 'v', 5: 'H', 6: 'F', 7: '8', 8: 'P', 9: 'Y', 10: 'a', 11: 'n', 12: 'j', 13: '2', 14: 'C', 15: 'd', 16: 'L', 17: 'U', 18: '9', 19: 'W', 20: '1', 21: '6', 22: 'y', 23: 'T', 24: 'p', 25: 'A', 26: 's', 27: 'I', 28: 'S', 29: 'O', 30: 'u', 31: 'k', 32: 'c', 33: ' ', 34: 'f', 35: 'm', 36: 'x', 37: 'b', 38: 'K', 39: 't', 40: 'h', 41: 'M', 42: 'e', 43: 'o', 44: 'w', 45: 'l', 46: 'i', 47: '4', 48: '5', 49: 'N', 50: 'r', 51: '7', 52: 'D', 53

In [55]:
# hyperparameters

# iteration = 1000
iteration = 5
sequence_length = 50
batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
hidden_size = 500  # size of hidden layer of neurons.  
learning_rate = 1e-1


# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1); initialization of previous state

In [56]:
def forwardprop(inputs, targets, h_prev):
        
    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

#         y_class = np.zeros((num_chars, 1)) 
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

    return loss, ps, hs, xs

In [57]:
def backprop(ps, inputs, hs, xs, targets):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

In [58]:
%%time

data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 

for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        
        inputs = [char_to_int[ch] for ch in sonnet_text[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in sonnet_text[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
        if (data_pointer+sequence_length+1 >= len(sonnet_text) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)
    
        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs, targets) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
        data_pointer += sequence_length # move data pointer
        
    if i % 100 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress

iter 0, loss: 18.432920
CPU times: user 42.7 s, sys: 4.05 s, total: 46.7 s
Wall time: 28.9 s


In [59]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = test_char + ''.join(int_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [61]:
predict('N', 30)

----
 No  waS en o fddcl eLto1ih eal  
----


Using Keras and LSTM

In [7]:
'''
#Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
**Notes**
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
import os, re
# from tensorflow.keras.datasets import imdb

# max_features = 20000
max_features = 110
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

In [29]:
data_files = os.listdir('./shakes')
files = []
for filename in data_files:
    if filename.startswith('sonnets'): # make sure only loading the text files
        path = os.path.join('./shakes', filename)
        with open(path, 'r') as f:
            content = f.read()
            content = re.sub('[^A-Za-z0-9]+', ' ', content) #elminate non-alphanumerics
            files.append(content)

In [30]:
files[0]

' 110 Alas tis true I have gone here and there And made my self a motley to the view Gored mine own thoughts sold cheap what is most dear Made old offences of affections new Most true it is that I have looked on truth Askance and strangely but by all above These blenches gave my heart another youth And worse essays proved thee my best of love Now all is done have what shall have no end Mine appetite I never more will grind On newer proof to try an older friend A god in love to whom I am confined Then give me welcome next my heaven the best Even to thy pure and most most loving breast '

In [36]:
file_text = []
file_text = [a[:50] for a in files]

In [37]:
print (f'Length of text: {len(file_text)} characters')

Length of text: 154 characters


In [38]:
print(file_text)

[' 110 Alas tis true I have gone here and there And ', ' 39 O how thy worth with manners may I sing When t', ' 47 Betwixt mine eye and heart a league is took An', ' 37 As a decrepit father takes delight To see his ', ' 109 O never say that I was false of heart Though ', ' 23 As an unperfect actor on the stage Who with hi', ' 2 When forty winters shall besiege thy brow And d', ' 85 My tongue tied muse in manners holds her still', ' 50 How heavy do I journey on the way When what I ', ' 115 Those lines that I before have writ do lie Ev', ' 25 Let those who are in favour with their stars O', ' 121 Tis better to be vile than vile esteemed When', ' Th expense of spirit in a waste of shame Is lust ', ' 62 Sin of self love possesseth all mine eye And a', ' 114 Or whether doth my mind being crowned with yo', ' 26 Lord of my love to whom in vassalage Thy merit', ' 100 Where art thou Muse that thou forget st so lo', ' 44 If the dull substance of my flesh were thought', ' 107 Not mine own fears no

In [35]:
file_text = " ".join(file_text)
print(file_text)

 110 Alas tis true I have gone here and there And   39 O how thy worth with manners may I sing When t  47 Betwixt mine eye and heart a league is took An  37 As a decrepit father takes delight To see his   109 O never say that I was false of heart Though   23 As an unperfect actor on the stage Who with hi  2 When forty winters shall besiege thy brow And d  85 My tongue tied muse in manners holds her still  50 How heavy do I journey on the way When what I   115 Those lines that I before have writ do lie Ev  25 Let those who are in favour with their stars O  121 Tis better to be vile than vile esteemed When  Th expense of spirit in a waste of shame Is lust   62 Sin of self love possesseth all mine eye And a  114 Or whether doth my mind being crowned with yo  26 Lord of my love to whom in vassalage Thy merit  100 Where art thou Muse that thou forget st so lo  44 If the dull substance of my flesh were thought  107 Not mine own fears nor the prophetic soul Of   125 Were t aught to me I bore 

In [ ]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN